# simulatorクラスの定義


PHYSBOではSimulator関数を使用してロス関数を定義する。
ここでは、一次元の関数の最小値を求める例題をもとに、simulatorクラスの定義方法について紹介する。

## 探索空間の定義

その他のチュートリアルと同じように、最初に関数を探索する空間を定義する。
以下の例では、探索空間``X``を ``x_min = -2.0``から``x_max = 2.0``まで``window_num=10001``分割で刻んだグリッドで定義している。
なお、``X``は ``window_num x d`` のndarray形式にする必要がある(``d``は次元数)。そのため、reshapeを行って変形している。

In [1]:
#In
import numpy as np
import scipy
import combo
import os
import urllib
import itertools

#In
#Create candidate
window_num=10001
x_max = 2.0
x_min = -2.0

X = np.linspace(x_min,x_max,window_num).reshape(window_num, 1)

## simulatorクラスの定義

ロス関数を定義するためのsimulatorクラスをここで定義する。

今回はf(x) = 3 x^4 + 4 x ^3 + 1.0 が最小となるxを探索するという問題設定にしている(答えはx=-1.0)。

simulatorクラスでは、``__call__``関数を定義する(初期変数などがある場合は``__init__``を定義する)。
actionは探索空間の中から取り出すグリッドのindex番号を示しており、複数の候補を一度に計算できるように一般的にndarrayの形式を取っている。
今回は一つの候補のみを毎回計算するため、``action_idx=action[0]``として``X``から候補点を一つ選んでいる。
PHYSBOではロス関数に-1をかけたものが最大となるものを求める仕様になっているため、``delta``に候補点でのf(x)の値をいれ、最後に-1をかけたものを返している。

In [2]:
# Declare the class for calling the simulator.
class simulator:

    def __call__(self, action ):
        action_idx = action[0]
        x = X[action_idx][0]
        delta = 3.0*x**4 + 4.0*x**3 + 1.0
        delta_list.append(delta)
        x_list.append(X[action_idx][0])

        print ("*********************")
        print ("Present optimum interactions")

        print ("x_opt=", x_list[np.argmin(np.array(delta_list))])

        return -delta, x

## ランダムサーチ

ベイズ最適化を行う前に、探索空間からランダムに点を求め事前にトレーニングデータを用意する(飛ばすことも可能)。

In [3]:
delta_list=[]
x_list = []
#In
# Design of policy
# Declaring the policy by
policy = combo.search.discrete.policy(test_X=X)
# test_X is the set of candidates which is represented by numpy.array.
# Each row vector represents the feature vector of the corresponding candidate

# set the seed parameter
policy.set_seed( 1 )


# If you want to perform the initial random search before starting the Bayesian optimization,
# the random sampling is performed by

res = policy.random_search(max_num_probes=50, simulator=simulator())
# Input:
# max_num_probes: number of random search
# simulator = simulator
# output: combo.search.discreate.results (class)

*********************
Present optimum interactions
x_opt= 1.9816000000000003
0001-th step: f(x) = -78.382728 (action=9954)
   current best f(x) = -78.382728 (best action=9954) 

*********************
Present optimum interactions
x_opt= 1.6812
0002-th step: f(x) = -43.973378 (action=9203)
   current best f(x) = -43.973378 (best action=9203) 

*********************
Present optimum interactions
x_opt= 0.31720000000000015
0003-th step: f(x) = -1.158032 (action=5793)
   current best f(x) = -1.158032 (best action=5793) 

*********************
Present optimum interactions
x_opt= 0.31720000000000015
0004-th step: f(x) = -8.451857 (action=7546)
   current best f(x) = -1.158032 (best action=5793) 

*********************
Present optimum interactions
x_opt= 0.31720000000000015
0005-th step: f(x) = -11.251199 (action=7793)
   current best f(x) = -1.158032 (best action=5793) 

*********************
Present optimum interactions
x_opt= -1.0048
0006-th step: f(x) = -0.000139 (action=2488)
   current be

## ベイズ最適化

定義したsimulatorに対してベイズ最適化を実施する。

In [4]:
# single query Bayesian search
# The single query version of COMBO is performed by
res = policy.bayes_search(max_num_probes= 150, simulator=simulator(), score='TS',
                                                  interval=20, num_rand_basis=2000)

# Input
# max_num_probes: number of searching by Bayesian optimization
# simulator: the class of simulator which is defined above
# score: the type of aquision funciton. TS, EI and PI are available
# interval: the timing for learning the hyper parameter.
#               In this case, the hyper parameter is learned at each 20 steps
#               If you set the negative value to interval, the hyper parameter learning is not performed
#               If you set zero to interval, the hyper parameter learning is performed only at the first step
# num_rand_basis: the number of basis function. If you choose 0,  ordinary Gaussian process runs

Start the initial hyper parameter searching ...
Done

Start the hyper parameter learning ...
0-th epoch, marginal likelihood 118.05664465110657
50-th epoch, marginal likelihood 115.61965314112352
100-th epoch, marginal likelihood 113.21685326201997
150-th epoch, marginal likelihood 110.8444321638101
200-th epoch, marginal likelihood 108.49802583494909
250-th epoch, marginal likelihood 106.17399199445914
300-th epoch, marginal likelihood 103.8692800425601
350-th epoch, marginal likelihood 101.5813087797087
400-th epoch, marginal likelihood 99.30786975273699
450-th epoch, marginal likelihood 97.04705031706425
500-th epoch, marginal likelihood 94.7971731817271
Done

 Parameters of Gaussian kernel 
 
 width  =  [3.]
 scale  =  1.0
 scale2 =  1.0
 

*********************
Present optimum interactions
x_opt= -1.0048
0051-th step: f(x) = -0.091016 (action=2214)
   current best f(x) = -0.000139 (best action=2488) 

*********************
Present optimum interactions
x_opt= -1.0048
0052-th step: 

*********************
Present optimum interactions
x_opt= -0.9967999999999999
0089-th step: f(x) = -0.000850 (action=2530)
   current best f(x) = -0.000061 (best action=2508) 

*********************
Present optimum interactions
x_opt= -0.9967999999999999
0090-th step: f(x) = -0.113555 (action=2183)
   current best f(x) = -0.000061 (best action=2508) 

Start the initial hyper parameter searching ...
Done

Start the hyper parameter learning ...
0-th epoch, marginal likelihood 176.68551959814403
50-th epoch, marginal likelihood 170.6090513388044
100-th epoch, marginal likelihood 165.04301936981406
150-th epoch, marginal likelihood 159.89087138340784
200-th epoch, marginal likelihood 155.1352654915216
250-th epoch, marginal likelihood 150.58043587480873
300-th epoch, marginal likelihood 146.16489032607686
350-th epoch, marginal likelihood 141.81386382240467
400-th epoch, marginal likelihood 137.48594038755525
450-th epoch, marginal likelihood 133.1690705298636
500-th epoch, marginal likeli

*********************
Present optimum interactions
x_opt= -0.9967999999999999
0127-th step: f(x) = -0.029584 (action=2332)
   current best f(x) = -0.000061 (best action=2508) 

*********************
Present optimum interactions
x_opt= -0.9967999999999999
0128-th step: f(x) = -0.064864 (action=2256)
   current best f(x) = -0.000061 (best action=2508) 

*********************
Present optimum interactions
x_opt= -0.9967999999999999
0129-th step: f(x) = -0.014898 (action=2629)
   current best f(x) = -0.000061 (best action=2508) 

*********************
Present optimum interactions
x_opt= -0.9967999999999999
0130-th step: f(x) = -0.002719 (action=2554)
   current best f(x) = -0.000061 (best action=2508) 

Start the initial hyper parameter searching ...
Done

Start the hyper parameter learning ...
0-th epoch, marginal likelihood 210.9205515823147
50-th epoch, marginal likelihood 197.26387997943272
100-th epoch, marginal likelihood 183.88121972721
150-th epoch, marginal likelihood 170.664937130

*********************
Present optimum interactions
x_opt= -0.9984
0165-th step: f(x) = -0.003569 (action=2562)
   current best f(x) = -0.000015 (best action=2504) 

*********************
Present optimum interactions
x_opt= -0.9984
0166-th step: f(x) = -0.009109 (action=2405)
   current best f(x) = -0.000015 (best action=2504) 

*********************
Present optimum interactions
x_opt= -0.9984
0167-th step: f(x) = -0.007408 (action=2590)
   current best f(x) = -0.000015 (best action=2504) 

*********************
Present optimum interactions
x_opt= -0.9984
0168-th step: f(x) = -0.008572 (action=2597)
   current best f(x) = -0.000015 (best action=2504) 

*********************
Present optimum interactions
x_opt= -0.9984
0169-th step: f(x) = -0.053865 (action=2754)
   current best f(x) = -0.000015 (best action=2504) 

*********************
Present optimum interactions
x_opt= -0.9984
0170-th step: f(x) = -0.004157 (action=2567)
   current best f(x) = -0.000015 (best action=2504) 

Start the 

## 結果表示

得られた結果のうち、一番よいスコアを持つものは``export_all_sequence_best_fx()``を用いることで取り出すことができる。
また、これまでの全ての履歴をみたい場合には、``chosed_actions``を呼び出すことで表示することができる。

In [10]:
#In
best_fx, best_action = res.export_all_sequence_best_fx()

print ("*********************")
print ('')
print ('Estimated model parameter')
print ('x=',X[int(best_action[-1])])
print ('')
print ('')

#In

# The result of searching is summarized in the class combo.search.discrete.results.history()
# res.fx: observed negative energy at each step
# res.chosed_actions: history of choosed actions
# fbest, best_action= res.export_all_sequence_best_fx(): current best fx and current best action
#                                                                                                   that has been observed until each step
# res.total_num_search: total number of search
print ('f(x)=')
print (res.fx[0:res.total_num_search])
print ('current best')
print (best_fx)
print ('current best action=')
print (best_action)
print ('history of chosed actions=')
print (res.chosed_actions[0:res.total_num_search])

*********************

Estimated model parameter
x= [-1.]


f(x)=
[-7.83827277e+01 -4.39733776e+01 -1.15803202e+00 -8.45185682e+00
 -1.12511987e+01 -1.39126329e-04 -2.27867548e+01 -2.01660963e-02
 -1.30066736e+01 -9.82035709e-01 -1.46828688e+01 -1.21715959e-01
 -8.84114971e-01 -3.83723611e+01 -2.50264142e-01 -5.60939838e+00
 -4.61217025e+01 -3.50118509e+01 -1.43212485e+00 -9.99541197e-01
 -8.91954429e-03 -1.95989753e+00 -1.54585793e+01 -1.11385356e+01
 -3.21547361e+00 -3.62095667e+00 -2.58093284e+00 -4.76653158e+00
 -1.38730345e+00 -8.40401743e+00 -9.46707366e-01 -5.85930620e-01
 -5.80594347e-01 -5.02515975e+01 -1.63518750e-01 -1.84226726e-01
 -1.22783425e+00 -6.95649296e+01 -2.04994116e+01 -9.65867289e-01
 -4.48898948e+01 -1.56168243e+00 -6.42280024e+01 -1.68319848e-01
 -6.09803118e-02 -1.12961222e+01 -4.85198093e+00 -7.45764240e+01
 -2.45410059e-01 -9.99624088e-01 -9.10155488e-02 -3.69520939e-01
 -6.04362561e-02 -3.11572178e-01 -2.06809575e-02 -6.11781706e-05
 -7.95476642e-02 -1.1124